In [1]:
# Assistant API(OpenAI)
# Agent와 비슷한 기능 -> Langchain 없이 OpenAI Python 패키지로 가능
"""
1. Assistant
 : Tool(Instructino) 과 OpenAI Model 로 조합된 AI Model
2. Thread
 : User와 Assistant가 주고 받는 메시지의 모음
 : User가 Message를 보내면 Thread에 전달되야 함.
 : 별도 메모리 관리를 할 것없이 OpenAI가 이를 관리하여 Message들을 Thread에 관리됨(Model의 context memory에 맞도록 history 관리)
3. Messages
 : User Message
4. Run 
 : Thread의 내용을 Model을 통해서 실행
"""
from langchain.tools import DuckDuckGoSearchResults
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchResults()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")

def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())

def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]


In [25]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-1106-preview", # https://platform.openai.com/playground?mode=assistant : Model List
#     tools=functions,
# )

assistant_id = "asst_HUHCXk0HEeHfarAzTKrt5f50"

In [26]:
# 1. Thread 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_uRQnChI1ATvKsc6Ax4w7mFez', created_at=1712844389, metadata={}, object='thread')

In [27]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )

# Run 생성 및 실행
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_wXGdnL5fzaPEJKJ0Xd05k29l', assistant_id='asst_HUHCXk0HEeHfarAzTKrt5f50', cancelled_at=None, completed_at=None, created_at=1712844391, expires_at=1712844991, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_uRQnChI1ATvKsc6Ax4w7mFez', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'type':

In [29]:
get_run(run.id, thread.id).status

'requires_action'

In [23]:
# Message를 입력된 순서대로 보여주게 Reverse
def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        # print(message)
        print(f"{message.role}: {message.content[0].text.value}")

get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy
